<a href="https://colab.research.google.com/github/Vishal152k/Vishal152k.github.io/blob/master/Sentiment_Analysis_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']
encoder = info.features['text'].encoder 

BUFFER_SIZE = 10000
BATCH_SIZE = 64

padded_shapes = ([None], ())

train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE,
                                                    padded_shapes = padded_shapes)
test_dataset = test_dataset.padded_batch(BATCH_SIZE,
                                                    padded_shapes = padded_shapes)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteB6I76Z/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteB6I76Z/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteB6I76Z/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

Epoch 1/10
391/391 [==============================] - 78s 199ms/step - loss: 0.6584 - accuracy: 0.5543 - val_loss: 0.4999 - val_accuracy: 0.7594
Epoch 2/10
391/391 [==============================] - 78s 200ms/step - loss: 0.4045 - accuracy: 0.8273 - val_loss: 0.3984 - val_accuracy: 0.8297
Epoch 3/10
391/391 [==============================] - 79s 202ms/step - loss: 0.2827 - accuracy: 0.8917 - val_loss: 0.3455 - val_accuracy: 0.8578
Epoch 4/10
391/391 [==============================] - 78s 200ms/step - loss: 0.2300 - accuracy: 0.9198 - val_loss: 0.3431 - val_accuracy: 0.8589
Epoch 5/10
391/391 [==============================] - 78s 199ms/step - loss: 0.1932 - accuracy: 0.9362 - val_loss: 0.3914 - val_accuracy: 0.8625
Epoch 6/10
391/391 [==============================] - 79s 201ms/step - loss: 0.1629 - accuracy: 0.9505 - val_loss: 0.4043 - val_accuracy: 0.8490
Epoch 7/10
391/391 [==============================] - 79s 201ms/step - loss: 0.1464 - accuracy: 0.9560 - val_loss: 0.4045 - val_ac

NameError: ignored

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 31s 80ms/step - loss: 0.5406 - accuracy: 0.8422
Test Loss: 0.5405912399291992
Test Accuracy: 0.8421599864959717


In [ ]:
def pad_to_size(vec, size):
  zeroes = [0]*(size - len(vec))
  vec.extend(zeroes)
  return vec

def sample_predict(sentence, pad):
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = loaded_model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return predictions

In [ ]:
from keras.models import model_from_json

model_json = model.to_json()
with open('model.json', 'w') as json_file:
  json_file.write(model_json)

model.save_weights('model.h5')

Using TensorFlow backend.


In [ ]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
import os

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
loaded_model.compile(loss= 'binary_crossentropy',
              optimizer='adam',
              metrics = ['accuracy'])


In [ ]:
sample_text = ("This is how our model works")
predictions = sample_predict(sample_text, pad=True)*100
print(predictions)

[[0.55715495]]


In [ ]:
# API scraping

import requests
url = ('http://newsapi.org/v2/top-headlines?'
       'country=us&'
       'apiKey=236cc88a8ca2492e8ba855bcfce9ef40')
response = requests.get(url)
response_json = response.json()

#print(type(response_json['totalResults']))

article = response_json["articles"]

title = []
description = []
url = []
url_image = []
prediction=[]

for ar in article: 
  title.append(ar["title"]) 
  description.append(ar["description"])
  url.append(ar["url"])
  url_image.append(ar["urlToImage"])
  prediction.append(sample_predict(ar["title"], pad=True)*100)                

#for i in range(response_json['totalResults']):
  # printing all trending news
  #print(i + 1, results[i])
  #print('title' + ':' + ' ' + title[i])
  #print('description' + ':' + ' ' + description[i])
  #print('url' + ':' + ' ' + url[i])
  #print('url_image' + ':' + ' ' + url_image[i])
  #print('prediction' + ':' + ' ' + prediction[i])         
#print(type(response_json))
#print(response_json)

In [ ]:
import pandas as pd

news_dict = pd.DataFrame()
news_dict['title'] = pd.Series(title)
news_dict['description'] = pd.Series(description)
news_dict['url'] = pd.Series(url)
news_dict['url_image'] = pd.Series(url_image)

In [ ]:
news_dict

In [ ]:
news_dict['prediction'] = pd.Series(prediction)

In [ ]:
news_dict

,title,description,url,url_image,prediction
0,Trump administration asks Supreme Court to str...,House Speaker Nancy Pelosi (D-Calif.) denounce...,https://www.washingtonpost.com/nation/2020/06/...,https://www.washingtonpost.com/wp-apps/imrs.ph...,[[0.95627445]]
1,European markets climb despite virus fears; Wi...,European markets advanced on Friday morning de...,https://www.cnbc.com/2020/06/26/europe-markets...,https://image.cnbcfm.com/api/v1/image/10658859...,[[0.36092347]]
2,Warning—Apple Suddenly Catches TikTok Secretly...,"If you are a TikTok user, you should be worrie...",https://www.forbes.com/sites/zakdoffman/2020/0...,https://thumbor.forbes.com/thumbor/fit-in/1200...,[[0.36978927]]
3,Trump's post-Covid bubble is popping - CNN,America's single worst day of new coronavirus ...,https://www.cnn.com/2020/06/26/politics/donald...,https://cdn.cnn.com/cnnnext/dam/assets/2006252...,[[0.3346045]]
4,"Trump calls protesters 'terrorists,' pledges '...",The president's comments come as demonstrators...,https://www.politico.com/news/2020/06/26/trump...,https://static.politico.com/e0/0b/87569d9b4245...,[[0.3090738]]
5,G-Eazy's 'Everything's Strange Here' Has Arriv...,,https://www.billboard.com/articles/news/940942...,https://static.billboard.com/files/media/G-Eaz...,[[0.8612207]]
6,“He’s a Savage But I’m a Killer”- Mickey Gall ...,UFC Welterweight fighter Mickey Gall will make...,https://www.essentiallysports.com/ufc-news-hes...,https://image-cdn.essentiallysports.com/wp-con...,[[0.6236033]]
7,Virgin Australia finds new owner in US private...,"Virgin Australia has found a buyer, US private...",https://www.cnn.com/2020/06/25/business/virgin...,https://cdn.cnn.com/cnnnext/dam/assets/2006252...,[[0.7586025]]
8,Bear-size fossil discovered in Australia is a ...,The newly discovered fossil of a prehistoric m...,https://www.cnet.com/news/mysterious-new-creat...,https://cnet4.cbsistatic.com/img/AFMagXRWClXfx...,[[0.34219244]]
9,FCC visit reveals that a key Pixel 4 feature w...,The Pixel 4a has yet to be introduced by Googl...,https://www.phonearena.com/news/pixel-4a-fcc-v...,https://i-cdn.phonearena.com/images/article/12...,[[0.44012523]]


In [ ]:
"""
title_positive = []
description_positive = []
url_positive = []
url_image_positive = []
prediction_positive =[]

title_negative = []
description_negative = []
url_negative = []
url_image_negative = []
prediction_negative =[]


for i in range(response_json['totalResults']):
  if(float(prediction[i])>=0.00):
    title_positive.append(title[i])
    description_positive.append(description[i])
    url_positive.append(url[i])
    url_image_positive.append(url_image_positive[i])
    prediction_positive.append(prediction[i])
  
  else:
    title_negative.append(title[i])
    description_negative.append(description[i])
    url_negative.append(url[i])
    url_image_negative.append(url_image_positive[i])
    prediction_negative.append(prediction[i]) 
"""

In [ ]:
"""
news_dict_positive = pd.DataFrame()
news_dict_positive['title'] = pd.Series(title_positive)
news_dict_positive['description'] = pd.Series(description_positive)
news_dict_positive['url'] = pd.Series(url_positive)
news_dict_positive['url_image'] = pd.Series(url_image_positive)
news_dict_positive['prediction'] = pd.Series(prediction_positive)

news_dict_negative = pd.DataFrame()
news_dict_negative['title'] = pd.Series(title_negative)
news_dict_negative['description'] = pd.Series(description_negative)
news_dict_negative['url'] = pd.Series(url_negative)
news_dict_negative['url_image'] = pd.Series(url_image_negative)
news_dict_negative['prediction'] = pd.Series(prediction_negative)
"""

In [ ]:
#news_dict_positive

In [ ]:
#news_dict_negative

In [ ]:
news_dict.to_excel('news.xlsx',index=False)

In [ ]:
import openpyxl as xl

wb = xl.load_workbook('news.xlsx')
sheet = wb['Sheet1']


row_count = sheet.max_row
#print(sheet.cell(2,1).value)

for i in range (2, (row_count + 1), 1):
  sheet.cell(i,1).hyperlink = sheet.cell(i,3).value
  sheet.cell(i,1).value = sheet.cell(i,1).value

for i in range (2, (row_count + 1), 1):
  description = sheet.cell(i,2).value
  if(type(description) == type('string')):
    sheet.cell(i,2).value = description[0:50] + '... read more'

  else: 
    pass 

wb.save('news_2.xlsx')